#Install MineRL

In [1]:
%%capture
!sudo add-apt-repository -y ppa:openjdk-r/ppa
!sudo apt-get purge openjdk-*
!sudo apt-get install openjdk-8-jdk
!sudo apt-get install xvfb xserver-xephyr vnc4server python-opengl ffmpeg libopenal-dev
!sudo apt-get upgrade
!pip3 install pyvirtualdisplay
!pip3 install -U colabgymrender
!pip install coloredlogs
!pip install daemoniker 
!pip install Pyro4
!pip install xmltodict
!pip install chainerrl
!pip install cupy
# !pip3 install --upgrade minerl

In [2]:
%%capture
!rm -r /content/minerl
!git clone https://github.com/minerllabs/minerl.git
%cd /content/minerl
!git checkout dev
%cd /content

In [3]:
!rm -r /usr/local/lib/python3.7/dist-packages/minerl
!cp -RT /content/minerl/minerl /usr/local/lib/python3.7/dist-packages/minerl

rm: cannot remove '/usr/local/lib/python3.7/dist-packages/minerl': No such file or directory


In [4]:
%%capture
%cd /content
!rm -r /content/baselines
!git clone https://github.com/Daetheys/baselines
!cp -r /content/baselines/2019/general/chainerrl/baselines /content/tmp
!rm -r /content/baselines
!mv /content/tmp /content/baselines
%cd /content/baselines/

In [5]:
import gym
import minerl
from tqdm.notebook import tqdm
from colabgymrender.recorder import Recorder
from pyvirtualdisplay import Display

import numpy as np
import matplotlib.pyplot as plt

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2555904/45929032 bytes (5.6%)5218304/45929032 bytes (11.4%)9199616/45929032 bytes (20.0%)13041664/45929032 bytes (28.4%)17006592/45929032 bytes (37.0%)20979712/45929032 bytes (45.7%)24862720/45929032 bytes (54.1%)28680192/45929032 bytes (62.4%)32620544/45929032 bytes (71.0%)36143104/45929032 bytes (78.7%)40321024/45929032 bytes (87.8%)44441600/45929032 bytes (96.8%)

In [6]:
display = Display(visible=0, size=(400, 300))
display.start();

#Env

In [7]:
from minerl.herobraine.env_specs.simple_embodiment import SimpleEmbodimentEnvSpec
from minerl.herobraine.hero.mc import MS_PER_STEP, STEPS_PER_MS
from minerl.herobraine.hero.handler import Handler
from typing import List

import minerl.herobraine
import minerl.herobraine.hero.handlers as handlers
from minerl.herobraine.env_spec import EnvSpec


In [8]:
%%capture
!git clone https://github.com/Daetheys/baselines
%cd baselines/2019/general/chainerrl/baselines/

In [9]:
import flat_treechop

In [10]:
from minerl.herobraine.env_specs.simple_embodiment import SimpleEmbodimentEnvSpec
from minerl.herobraine.hero.mc import MS_PER_STEP, STEPS_PER_MS
from minerl.herobraine.hero.handler import Handler
from typing import List

import minerl.herobraine
import minerl.herobraine.hero.handlers as handlers
from minerl.herobraine.env_spec import EnvSpec

TREECHOP_DOC = """
"""
INFTY = 2147483647
class Flat(SimpleEmbodimentEnvSpec):
    def __init__(self, *args, **kwargs):
        if 'name' not in kwargs:
            kwargs['name'] = 'MineRLTreechopEcoFlat-v1'

        self.__time = 0

        super().__init__(*args,
                         max_episode_steps=INFTY, reward_threshold=INFTY,
                         **kwargs)

    def create_actionables(self) -> List[Handler]:
        return super().create_actionables() + [
            handlers.EquipAction(["diamond_axe"]),
            handlers.ChatAction()
        ]

    def create_rewardables(self) -> List[Handler]:
        return [
            handlers.RewardForCollectingItems([
                dict(type="log", amount=1, reward=1.0),
            ])
        ]

    def create_agent_start(self) -> List[Handler]:
        return [
            handlers.SimpleInventoryAgentStart([
                dict(type="diamond_axe", quantity=1)
            ]),
            handlers.AgentStartPlacement(0, 4, 0, 0, 0)
        ]

    def create_observables(self):
        return super().create_observables() + [
         handlers.ObservationFromCurrentLocation()
        ]

    def create_agent_handlers(self) -> List[Handler]:
        return []

    def create_agent_handlers(self) -> List[Handler]:
        return [
            handlers.AgentQuitFromPossessingItem([
                dict(type="log", amount=INFTY)]
            )
        ]

    def create_server_world_generators(self) -> List[Handler]:
        return [
            #handlers.DefaultWorldGenerator(force_reset="true",generator_options=TREECHOP_WORLD_GENERATOR_OPTIONS)
            handlers.FlatWorldGenerator(generatorString='1;7,2;4')
        ]

    def create_server_quit_producers(self) -> List[Handler]:
        return [
            handlers.ServerQuitFromTimeUp(
                (INFTY)),
            handlers.ServerQuitWhenAnyAgentFinishes()
        ]

    def create_server_decorators(self) -> List[Handler]:
        return []

    def create_server_initial_conditions(self) -> List[Handler]:
        return [
            handlers.TimeInitialCondition(
                allow_passage_of_time=False
            ),
            handlers.SpawningInitialCondition(
                allow_spawning=False
            )
        ]

    def determine_success_from_rewards(self, rewards: list) -> bool:
        return sum(rewards) >= self.reward_threshold

    def is_from_folder(self, folder: str) -> bool:
        return folder == 'survivaltreechop'

    def get_docstring(self):
        return TREECHOP_DOC

try:
    env_abs = Flat()
    env_abs.register()
except gym.error.Error:
    pass

In [11]:
env = gym.make('MineRLTreechopEcoFlat-v1')

In [12]:
env.observation_space,env.action_space

(Dict(location_stats:Dict(biome_id:Box(low=0, high=167, shape=()), biome_rainfall:Box(low=0.0, high=1.0, shape=()), biome_temperature:Box(low=0.0, high=1.0, shape=()), can_see_sky:Box(low=0, high=1, shape=()), is_raining:Box(low=0, high=1, shape=()), light_level:Box(low=0, high=15, shape=()), pitch:Box(low=-180.0, high=180.0, shape=()), sea_level:Box(low=0, high=255, shape=()), sky_light_level:Box(low=0.0, high=1.0, shape=()), sun_brightness:Box(low=0.0, high=1.0, shape=()), xpos:Box(low=-640000.0, high=640000.0, shape=()), yaw:Box(low=-180.0, high=180.0, shape=()), ypos:Box(low=-640000.0, high=640000.0, shape=()), zpos:Box(low=-640000.0, high=640000.0, shape=())), pov:Box(low=0, high=255, shape=(64, 64, 3))),
 Dict(attack:Discrete(2), back:Discrete(2), camera:Box(low=-180.0, high=180.0, shape=(2,)), chat:Text(1,), equip:Enum(diamond_axe,none,other), forward:Discrete(2), jump:Discrete(2), left:Discrete(2), right:Discrete(2), sneak:Discrete(2), sprint:Discrete(2)))

In [13]:
def get_tree(x, z, h, y=2):
    '''
    h > 4
    '''
    res = []
    
    for i in range(y+h-1, y+h+1):
      for j in range(x-1, x+2):
        for k in range(z-1, z+2):
          res.append(f"/setblock {j} {i} {k} leaves\n")
    for i in range(y+h-3, y+h-1):
      for j in range(x-2, x+3):
        for k in range(z-2, z+3):
          res.append(f"/setblock {j} {i} {k} leaves\n")
    for i in range(y, y+h):
      res.append(f"/setblock {x} {i} {z} log\n")
    return res

In [14]:
def noop_actions ():
  return {
    'attack':0,            # Discrete(2)
    'back':0,              # Discrete(2)
    'camera':(0.0, 0.0),   # Box(low=-180.0, high=180.0, shape=(2,))
    'equip':'diamond_axe', # Enum(diamond_axe,none,other)
    'forward':0,           # Discrete(2)
    'jump':0,              # Discrete(2)
    'left':0,              # Discrete(2)
    'right':0,             # Discrete(2)
    'sneak':0,             # Discrete(2)
    'sprint':0,            # Discrete(2)
    'chat':''              # Text(1,)
   }

In [15]:
import copy
class EcologicalWrapper(gym.Wrapper):
    def __init__(self,env):
        super().__init__(env)
        self.observation_space = gym.spaces.Dict({'pov':self.env.observation_space['pov']})
        d = {}
        for k in self.env.action_space:
            if not(k in ['chat','equip']):
                d[k] = self.env.action_space[k]
        self.action_space = gym.spaces.Dict(d)

        self.dist_max = 12
        self.nb_trees = 3
        self.spawn_radius = 8
        self.iter = 50
    def step(self,a):
        a['chat'] = ''
        a['equip'] = 'diamond_axe'
        o,r,d,i = self.env.step(a)
        dist = ( (o['location_stats']['xpos']-self.get_x())**2 + (o['location_stats']['zpos']-self.get_z())**2 )**0.5
        #print(dist,o['location_stats']['xpos'],self.get_x())
        if dist>self.dist_max:
            d = True
        return o,r,d,i

    def get_x(self):
        return (self.iter-1)*50

    def get_z(self):
        return (self.iter-1)*50

    def spawn_trees(self):
        for i in range(self.nb_trees):
            x = 0;z = 0
            while -2 < x < 2 and -2 < z < 2:
                x = np.random.randint(-self.spawn_radius,self.spawn_radius)
                z = np.random.randint(-self.spawn_radius,self.spawn_radius)
            o = self.spawn_tree(x,z)
        return o
        
    def spawn_tree(self,x,z):
        h = np.random.randint(5,7)
        cmds = get_tree(x+self.get_x(),z+self.get_z(),h)
        for c in cmds:
            a = noop_actions()
            a['chat'] = c
            o,_,_,_ = self.env.step(a)
        return o

    def spawn_player(self):
        x = self.get_x()
        z = self.get_z()
        c = f'/tp @p {x} 4 {z} 0 0\n'

        a = noop_actions()
        a['chat'] = c
        self.env.step(a)

    def reset(self):
        if self.iter >= 50:
            self.iter = 0
            o = self.env.reset()
        self.iter += 1
        self.spawn_player()
        o = self.spawn_trees()
        return o



# ChaineRL

In [21]:
%%capture
!pip install chainerrl

In [22]:
import chainer
import chainerrl

chainerrl.misc.set_random_seed(42)

gamma = 0.99

def phi(x):
    # observation -> NN input
    return np.asarray(x)

update_interval = 1024
minibatch_size=32
epochs=3
clip_eps=0.1
standardize_advantages=False

outdir = './results'

lr = 2.5e-4
adam_eps = 1e-8

eval_n_runs = 3
eval_interval = (2000//4)*35

frame_skip = 4
frame_stack = 4

clip_eps = 0.1

always_keys = ['attack']
reverse_keys = ['forward']
exclude_keys = ['back','left','right','sneak','sprint']
exclude_noop = []

weight_decay = 0

steps = 8000000

gpu = 0
logging_level = 20

allow_pitch = True

In [23]:
import logging
import os

outdir = chainerrl.experiments.prepare_output_dir(None, outdir)

log_format = '%(levelname)-8s - %(asctime)s - [%(name)s %(funcName)s %(lineno)d] %(message)s'
logging.basicConfig(filename=os.path.join(outdir, 'log.txt'), format=log_format, level=logging_level)
console_handler = logging.StreamHandler()
console_handler.setLevel(logging_level)
console_handler.setFormatter(logging.Formatter(log_format))
logging.getLogger('').addHandler(console_handler)  # add hander to the root logger

from logging import getLogger
logger = getLogger(__name__)
logger.info('Output files are saved in {}'.format(outdir))

import utils
utils.log_versions()

INFO     - 2022-01-28 01:16:01,360 - [__main__ <module> 15] Output files are saved in ./results/20220128T011600.774836
INFO     - 2022-01-28 01:16:01,713 - [utils log_versions 9] 3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0]
INFO     - 2022-01-28 01:16:01,937 - [utils log_versions 10] absl-py==1.0.0,alabaster==0.7.12,albumentations==0.1.12,altair==4.2.0,appdirs==1.4.4,argon2-cffi==21.3.0,argon2-cffi-bindings==21.2.0,arviz==0.11.4,astor==0.8.1,astropy==4.3.1,astunparse==1.6.3,atari-py==0.2.9,atomicwrites==1.4.0,attrs==21.4.0,audioread==2.1.9,autograd==1.3,Babel==2.9.1,backcall==0.2.0,beautifulsoup4==4.6.3,bleach==4.1.0,blis==0.4.1,bokeh==2.3.3,Bottleneck==1.3.2,branca==0.4.2,bs4==0.0.1,CacheControl==0.12.10,cached-property==1.5.2,cachetools==4.2.4,catalogue==1.0.0,certifi==2021.10.8,cffi==1.15.0,cftime==1.5.2,chainer==7.8.1,chainerrl==0.8.0,chardet==3.0.4,charset-normalizer==2.0.10,click==7.1.2,cloudpickle==1.3.0,cmake==3.12.0,cmdstanpy==0.9.5,colabgymrender==1.0.9,colorcet==3.0.

In [24]:
from env_wrappers import (
    SerialDiscreteActionWrapper, CombineActionWrapper, SerialDiscreteCombineActionWrapper,
    ContinuingTimeLimitMonitor,
    MoveAxisWrapper, FrameSkip, ObtainPoVWrapper, PoVWithCompassAngleWrapper, GrayScaleWrapper)
from chainerrl.wrappers import ContinuingTimeLimit
from chainerrl.wrappers.atari_wrappers import FrameStack, ScaledFloatFrame
import os
import numpy as np

env = gym.make('MineRLTreechopFlat-v0')
def wrap_env(env,nb_step=8000,rotmax=None,outdir=outdir,test=False,eco=False):
    env = env.env
    max_episode_steps = nb_step
    if eco:
        env = EcologicalWrapper(env)
    env = ContinuingTimeLimit(env, max_episode_steps=max_episode_steps)

    #env = RotMax(env,rotmax=rotmax)
    env = ContinuingTimeLimitMonitor(
                    env, os.path.join(outdir, 'monitor'),
                    mode='evaluation' if test else 'training', video_callable=lambda episode_id: episode_id%10==0)

    env = FrameSkip(env, frame_skip)

    env = ObtainPoVWrapper(env)

    env = MoveAxisWrapper(env, source=-1, destination=0)  # convert hwc -> chw as Chainer requires.
    env = ScaledFloatFrame(env)

    env = FrameStack(env, frame_stack, channel_order='chw')

    env = SerialDiscreteActionWrapper(env,always_keys=always_keys, reverse_keys=reverse_keys, exclude_keys=exclude_keys, exclude_noop=exclude_noop,allow_pitch=allow_pitch,num_camera_discretize=3,max_camera_range=3)
    return env
env = wrap_env(env)

INFO     - 2022-01-28 01:16:01,975 - [env_wrappers __init__ 483] always pressing keys: ['attack']
INFO     - 2022-01-28 01:16:01,976 - [env_wrappers __init__ 489] reversed pressing keys: ['forward']
INFO     - 2022-01-28 01:16:01,978 - [env_wrappers __init__ 494] always ignored keys: ['back', 'left', 'right', 'sneak', 'sprint']
INFO     - 2022-01-28 01:16:01,980 - [env_wrappers __init__ 553] Dict(attack:Discrete(2), back:Discrete(2), camera:Box(low=-180.0, high=180.0, shape=(2,)), forward:Discrete(2), jump:Discrete(2), left:Discrete(2), right:Discrete(2), sneak:Discrete(2), sprint:Discrete(2)) is converted to Discrete(11).


In [25]:
from q_functions import NatureDQNHead, A3CFF
def parse_arch(arch, n_actions, n_input_channels):
    if arch == 'nature':
        head = NatureDQNHead(n_input_channels=n_input_channels, n_output_channels=512)
    else:
        raise RuntimeError('Unsupported architecture name: {}'.format(arch))
    return A3CFF(n_actions, head)

In [26]:
model = parse_arch('nature',env.action_space.n,env.observation_space.shape[0])

In [27]:
opt = chainer.optimizers.Adam(alpha=lr, eps=adam_eps)
opt.setup(model)
opt.add_hook(chainer.optimizer.GradientClipping(40))
if weight_decay > 0:
    opt.add_hook(chainerrl.optimizers.nonbias_weight_decay.NonbiasWeightDecay(weight_decay))

In [28]:
agent = chainerrl.agents.ppo.PPO(
        model, opt, gpu=gpu, gamma=gamma, phi=phi, update_interval=update_interval,
        minibatch_size=32, epochs=epochs, clip_eps=clip_eps, standardize_advantages=standardize_advantages)

In [29]:
# Linearly decay the learning rate to zero
def lr_setter(env, agent, value):
    agent.optimizer.alpha = value

lr_decay_hook = chainerrl.experiments.LinearInterpolationHook(
    steps, lr, 0, lr_setter)

# Linearly decay the clipping parameter to zero
def clip_eps_setter(env, agent, value):
    agent.clip_eps = max(value, 1e-8)

clip_eps_decay_hook = chainerrl.experiments.LinearInterpolationHook(
    steps, clip_eps, 0, clip_eps_setter)

In [31]:
!rm -r ./results

In [ ]:
for env_name,rotmax,nb_ep,eco in [('MineRLTreechopFlat-v0',None,200,False),('MineRLTreechop-v0',None,200,False)]:
    print('\n\n\n\n\n'+env_name+'\n\n\n\n')
    steps = (2000//4)*nb_ep
    eval_interval = (2000//4)*29
    outdir = './results_'+env_name+str(rotmax)
    outdir = chainerrl.experiments.prepare_output_dir(None, outdir)
    expl_env = wrap_env(gym.make(env_name),nb_step=2000,rotmax=rotmax,outdir=outdir,eco=eco)
    eval_env = wrap_env(gym.make(env_name),nb_step=8000,rotmax=rotmax,outdir=outdir,test=True,eco=eco)
    chainerrl.experiments.train_agent_with_evaluation(
            agent=agent, env=expl_env, steps=steps,
            eval_n_steps=None, eval_n_episodes=eval_n_runs, eval_interval=eval_interval,
            outdir=outdir, eval_env=eval_env,
            step_hooks=[lr_decay_hook, clip_eps_decay_hook],
            save_best_so_far_agent=True,
            #train_max_episode_len=2000,
            #eval_max_episode_len=2000
        ) 